In [2]:
import pandas as pd
import numpy as np
import os

from prophet import Prophet
import holidays

import matplotlib.pyplot as plt

Importing plotly failed. Interactive plots will not work.


In [3]:
files = os.listdir("cycling_Auckland")
files = [v for v in files if 'count' in v]


In [4]:
# google searches: Multiple files pandas concat
# Python aggrgate timestamp per day

data_list = []
for file in files:
    current_file = pd.read_csv(os.path.join("cycling_Auckland", file))
    data_list.append(current_file[['datetime', 'Tamaki Drive EB']])

Tamaki = pd.concat(data_list) 

Tamaki['datetime'] = pd.to_datetime(Tamaki['datetime'])
Tamaki['datetime'] = pd.to_datetime(Tamaki['datetime'])
Tamaki = Tamaki.sort_values(by = 'datetime')
Tamaki['dag'] = Tamaki['datetime'].dt.date
Tamaki = pd.DataFrame(Tamaki.groupby('dag', as_index = False).sum())

Tamaki

KeyError: "None of [Index(['datetime', 'Tamaki Drive EB'], dtype='object')] are in the [columns]"

In [ ]:
temp_files = ['temp_day.csv', 'temp_morning.csv','temp_commute.csv']
temp_list = []
for file in temp_files:
    current_file = pd.read_csv(os.path.join("weather/hourly/commute", file))
    #print(current_file.head())
    temp_list.append(current_file.iloc[:,[0,4]])
    
temp = pd.concat(temp_list) 
temp.columns = ["day", "temp"]

temp['day'] = pd.to_datetime(temp['day'])
temp = pd.DataFrame(temp.groupby('day', as_index = False).max())    
temp

In [ ]:
wind_files = ['wind_day.csv', 'wind_morning.csv','wind_commute.csv']
wind_list = []
for file in wind_files:
    current_file = pd.read_csv(os.path.join("weather/hourly/commute", file))
    #print(current_file.head())
    wind_list.append(current_file.iloc[:,[0,2]])
    
wind = pd.concat(wind_list) 
wind.columns = ["day", "wind"]

wind['day'] = pd.to_datetime(wind['day'])
wind = pd.DataFrame(wind.groupby('day', as_index = False).max())    
wind

In [ ]:
rain_files = ['rain_morning.csv', 'rain_commute.csv','rain_day.csv']
rain_list = []
for file in rain_files:
    current_file = pd.read_csv(os.path.join("weather/hourly/commute", file))
    #print(current_file.head())
    rain_list.append(current_file)
    
rain = pd.concat(rain_list) 
rain.columns = ["day", "rain"]

rain['day'] = pd.to_datetime(rain['day'])
rain = pd.DataFrame(rain.groupby('day', as_index = False).sum())    
rain

In [ ]:
sun_files = ['sun_day.csv', 'sun_commute.csv','sun_morning.csv']
sun_list = []
for file in sun_files:
    current_file = pd.read_csv(os.path.join("weather/hourly/commute", file))
    #print(current_file.head())
    sun_list.append(current_file)
    
sun = pd.concat(sun_list) 
sun
sun.columns = ["day", "sun"]

sun['day'] = pd.to_datetime(sun['day'])
sun = pd.DataFrame(sun.groupby('day', as_index = False).sum())    
sun

In [ ]:
all_weather = sun
all_weather = all_weather.merge(wind)
all_weather = all_weather.merge(rain)
all_weather = all_weather.merge(temp)
all_weather

In [ ]:
Tamaki_weather = Tamaki
Tamaki_weather['dag'] = pd.to_datetime(Tamaki_weather['dag'])
Tamaki_weather = Tamaki_weather[np.logical_and(Tamaki_weather['dag'] >= all_weather ['day'].min(),
                                              Tamaki_weather['dag'] <= all_weather ['day'].max()) ]
Tamaki_weather = Tamaki_weather.reset_index(drop=True)
Tamaki_weather

In [ ]:
if (all(Tamaki_weather['dag'] == all_weather['day'])):
    print("days match up, merging weather and cyclist counter data")
    Tamaki_merged  = pd.concat([Tamaki_weather , all_weather[["sun", "wind", "rain", "temp"]]], axis=1)
    
Tamaki_merged

In [ ]:
Tamaki_merged.columns = ['ds', 'y', 'sun', 'wind', 'rain', 'temp']

Tamaki_train = Tamaki_merged[Tamaki_merged['ds'] < '2018']
Tamaki_test = Tamaki_merged[Tamaki_merged['ds'] >= '2018']

In [ ]:
m = Prophet()
m.add_regressor('sun')
m.add_regressor('wind')
m.add_regressor('rain')
m.add_regressor('temp')
m.fit(Tamaki_train)


In [ ]:
forecast_days = 152
future = m.make_future_dataframe(periods=forecast_days )
future  = future.merge(Tamaki_merged[['ds', 'sun', 'wind', 'rain', 'temp']])
future

In [ ]:
forecast = m.predict(future)
forecast['truth'] = Tamaki_merged['y']

In [ ]:
forecast

In [ ]:
forecast_plot = forecast[forecast['ds'] >= '2018']

plt.figure(figsize = (10,7))
plt.plot(forecast_plot['ds'], forecast_plot['yhat'], label='Forecast', zorder = 1)
plt.plot(forecast_plot['ds'], forecast_plot['truth'], label='True cyclists', zorder = 0, color = 'orange')                                                                                       
plt.fill_between(forecast_plot['ds'], forecast_plot['yhat_upper'], forecast_plot['yhat_lower'],
                 alpha=.2, label='Confidence')
plt.title('2018 auckland forecast')
plt.legend()
plt.show()

In [ ]:
RMSE = np.sqrt(sum(np.square(forecast_plot['yhat'] - forecast_plot['truth']))/len(forecast_plot['yhat']))
RMSE

# including holiday

In [ ]:
holiday_DF = pd.DataFrame(holidays.NZ(prov='AUK', years=np.arange(2012, 2019)).items(), columns=['ds', 'holiday'])
holiday_DF.sort_values("ds", inplace = True)
holiday_DF

In [ ]:
m = Prophet(holidays = holiday_DF)
m.add_regressor('sun')
m.add_regressor('wind')
m.add_regressor('rain')
m.add_regressor('temp')
m.fit(Tamaki_train[Tamaki_train['ds'] >="2015"])

forecast_days = 152
future = m.make_future_dataframe(periods=forecast_days )

future  = future.merge(Tamaki_merged.loc[Tamaki_merged['ds'] >= "2015",['ds', 'sun', 'wind', 'rain', 'temp']])
future

forecast = m.predict(future)
forecast['truth'] = Tamaki_merged.loc[Tamaki_merged['ds'] >= "2015",'y'].reset_index(drop = True)


forecast_plot = forecast[forecast['ds'] >= '2018']

plt.figure(figsize = (10,7))
plt.plot(forecast_plot['ds'], forecast_plot['yhat'], label='Forecast', zorder = 1)
plt.plot(forecast_plot['ds'], forecast_plot['truth'], label='True cyclists', zorder = 0, color = 'orange')                                                                                       
plt.fill_between(forecast_plot['ds'], forecast_plot['yhat_upper'], forecast_plot['yhat_lower'],
                 alpha=.2, label='Confidence')
plt.title('2018 auckland forecast')
plt.legend()
plt.show()

In [ ]:
RMSE = np.sqrt(sum(np.square(forecast_plot['yhat'] - forecast_plot['truth']))/len(forecast_plot['yhat']))
RMSE